<center><h1>CSCI 4140: Natural Language Processing</h1></center>
<center><h1>CSCI/DASC 6040: Computational Analysis of Natural Languages</h1></center>

<center><h6>Spring 2025</h6></center>
<center><h6>Homework 1 - N-gram models</h6></center>
<center><h6>Due Sunday, January 26, at 11:59 PM</h6></center>

<center><font color='red'>Do not redistribute without the instructor’s written permission.</font></center>

The learning goals of this assignment are to:

- Understand how to compute language model probabilities using maximum likelihood estimation.
- Implement back-off.
- Have fun using a language model to probabilistically generate texts.

# N-gram Language model
- For undergraduates: **100 pts + 10 extra credit pts**
- For graduates: **110 pts**

## Preliminaries

In [1]:
import numpy as np
from math import log, exp
from collections import defaultdict, Counter
import random

We'll start by loading the data. The [WikiText language modeling dataset](https://huggingface.co/datasets/Salesforce/wikitext) is a collection of tokens extracted from the set of verified Good and Featured articles on Wikipedia.

In [2]:
DATA = {'test': '', 'train': '', 'valid': ''}

for data_split in DATA:
    fname = "tokens/wiki.{}.tokens".format(data_split)
    with open(fname, 'r') as f_wiki:
        DATA[data_split] = f_wiki.read().lower().split()

VOCAB = list(set(DATA['train']))

Now have a look at the data by running this cell.

In [ ]:
global DATA, VOCAB

print('train : %s ...' % DATA['train'][:10])
print('dev : %s ...' % DATA['valid'][:10])
print('test : %s ...' % DATA['test'][:10])
print('first 10 words in vocab: %s' % VOCAB[:10])

## Q1: Train N-gram language model (60 pts)

Complete the following `train_ngram_lm` function based on the following input/output specifications. If you've done it right, you should pass the tests in the cell below.

*Input:*
+ **data**: the data object created in the cell above that holds the tokenized Wikitext data
+ **order**: the order of the model (i.e., the "n" in "n-gram" model). If order=3, we compute $p(w_2 | w_0, w_1)$.

*Output:*
+ **lm**: A dictionary where the key is the history and the value is a probability distribution over the next word computed using the maximum likelihood estimate from the training data. Importantly, this dictionary should include *backoff* probabilities as well; e.g., for order=4, we want to store $p(w_3 | w_0,w_1,w_2)$ as well as $p(w_3|w_1,w_2)$ and $p(w_3|w_2)$. 

Each key should be a single string where the words that form the history have been concatenated using spaces. Given a key, its corresponding value should be a dictionary where each word type in the vocabulary is associated with its probability of appearing after the key. For example, the entry for the history 'w1 w2' should look like:

    
    lm['w1 w2'] = {'w0': 0.001, 'w1' : 1e-6, 'w2' : 1e-6, 'w3': 0.003, ...}
    
In this example, we also want to store `lm['w2']` and `lm['']`, which contain the bigram and unigram distributions respectively.

*Hint:* You might find the **defaultdict** and **Counter** classes in the **collections** module to be helpful.

In [3]:
def _backoff(lm: list[str], h: str) -> str:
    """
    Backoff until we find a history in lm or reach an empty string.
    
    Parameters
    ----------
    lm : str
        The trained n-gram language model.
    h : str
        The history string (context) to look up in the model.

    Returns
    -------
    str
        The longest found history in lm, or an empty string if no match is found.
    """
    
    while h not in lm and h:
            h = " ".join(h.split()[:-1]) # backoff n-1
    return h

def train_ngram_lm(data: list[str], n: int = 3) -> dict[str, dict[str, float]]:
    """ 
    Trains an n-gram language model with backoff support.

    Parameters:
    -----------
    data : list of str
        A list of tokens representing the training dataset.
    n : int, optional
        The order of the n-gram model. Defaults to 3.

    Returns:
    --------
    dict of {str: dict of {str: float}}
        A dictionary where:
        - Keys are space-separated strings representing the history (n - 1 context).
        - Values are dictionaries mapping possible next words to their probabilities.
    """

    n -= 1
    data = (['<S>'] * n) + data
    lm = defaultdict(Counter)

    # collect n..0-grams
    for i in range(len(data) - n): # up to len(data - n)
        for j in range (n, -1, -1):

            h, w = ' '.join(data[i: i + j]), data[i + n] 
            lm[h][w] += 1

    # convert raw counts into MLEs
    lm = { 
        h : {w: count / sum(word_counts.values()) for w, count in word_counts.items()} 
        for h, word_counts in lm.items() 
    }
    return lm

In [ ]:
def test_ngram_lm():
    import random
    global DATA
  
    print('checking empty history ...')
    lm1 = train_ngram_lm(DATA['train'], n = 1)
    assert '' in lm1, "empty history should be in the language model!"
    
    print('checking probability distributions ...')
    lm2 = train_ngram_lm(DATA['train'], n = 2 )
    sample = [sum(lm2[k].values()) for k in random.sample(list(lm2), 10)]
    assert all([a > 0.999 and a < 1.001 for a in sample]), "lm[history][word] should sum to 1!"
    
    print('checking lengths of histories ...')
    lm3 = train_ngram_lm(DATA['train'], n = 3)
    assert len(set([len(k.split()) for k in list(lm3)])) == 3, "lm object should store histories of all sizes!"
    
    print('checking word distribution values ...')
    assert lm1['']['the'] < 0.064 and lm1['']['the'] > 0.062 and \
           lm2['the']['first'] < 0.017 and lm2['the']['first'] > 0.016 and \
           lm3['the first']['time'] < 0.106 and lm3['the first']['time'] > 0.105, \
           "values do not match!"
    
    print("Congratulations, you passed the ngram check!")
    
test_ngram_lm()

## Q2: Generate text from n-gram language model (40 pts)

Complete the following `generate_text` function based on these input/output requirements:

*Input:*

+ **lm**: the lm object is the dictionary you return from  the **train_ngram_lm** function
+ **vocab**: vocab is a list of unique word types in the training set, already computed for you during data loading.
+ **context**: the input context string that you want to condition your language model on, should be a space-separated string of tokens
+ **order**: order of your language model (i.e., "n" in the "n-gram" model)
+ **num_tok**: number of tokens to be generated following the input context


*Output:*

+ generated text, should be a space-separated string
    
*Hint:*

After getting the next-word distribution given history, try using **[numpy.random.choice](https://numpy.org/doc/stable/reference/random/generated/numpy.random.choice.html)** to sample the next word from the distribution.

In [6]:

def generate_text(lm: dict[str, dict[str, float]], vocab: list[str], context: str = "he is the", n: int = 3, n_tokens: int = 25) -> str:
    """
    Generates text using an n-gram language model with backoff support.

    Parameters
    ----------
    lm : dict[str, dict[str, float]]
        The trained n-gram language model where:
        - Keys are space-separated history strings (n-1 context).
        - Values are dictionaries mapping next words to their probabilities.
    vocab : list[str]
        A list of unique words from the training dataset, used for fallback sampling.
    context : str, optional
        The initial text to seed the text generation process. Defaults to "he is the".
    n : int, optional
        The order of the n-gram model. Defaults to 3 (trigram model).
    n_tokens : int, optional
        The number of tokens to generate. Defaults to 25.

    Returns
    -------
    str
        A space-separated string representing the generated text.
    """

    n -= 1
    history = context.split()[:n]
    out = context.split()
    
    for _ in range(n_tokens):
        h = _backoff(lm, " ".join(history)) # backoff (n - 1) if history isn't found in lm

        # convert dict keys and values to arrays for np.random.choice sampling
        if h in lm:
            a, p = zip(*lm[h].items()) # w = [...] p = [...]
            w = np.random.choice(a = a, p = p)
        else:
            w = np.random.choice(a = vocab) # fallback to random selection
         
        out.append(w) # append to growing sequence 
        history = (history + [w])[-n:]
    return " ".join(out) # return full generated text

Now try to generate some texts, generated by ngram language model with different orders.

In [ ]:
global DATA, VOCAB
order = 1
print(f"[{order}-GRAM]: {generate_text(lm = train_ngram_lm(data = DATA['train'], n = order), vocab = VOCAB, context = 'he is the', n = order)}")

In [ ]:
global DATA, VOCAB
order = 2
print(f"[{order}-GRAM]: {generate_text(lm = train_ngram_lm(data = DATA['train'], n = order), vocab = VOCAB, context = 'he is the', n = order)}")

In [ ]:
global DATA, VOCAB
order = 3
print(f"[{order}-GRAM]: {generate_text(lm = train_ngram_lm(data = DATA['train'], n = order), vocab = VOCAB, context = 'he is the', n = order)}")

In [ ]:
global DATA, VOCAB
order = 4
print(f"[{order}-GRAM]: {generate_text(lm = train_ngram_lm(data = DATA['train'], n = order), vocab = VOCAB, context='he is the', n = order)}")

## Q3 : Evaluate the models (10 pts)
Now let's evaluate the models quantitively using the intrinsic metric **perplexity**. 

Recall perplexity is the inverse probability of the test text
$$\text{PP}(w_1, \dots, w_t) = P(w_1, \dots, w_t)^{-\frac{1}{T}}$$

For an n-gram model, perplexity is computed by
$$\text{PP}(w_1, \dots, w_t) = \left[\prod_{t=1}^T P(w_t|w_{t-1},\ldots,w_{t-n+1})\right]^{-\frac{1}{T}}$$

To address the numerical issue (underflow), we usually compute
$$\text{PP}(w_1, \dots, w_t) = \exp\left(-\frac{1}{T}\sum_i \log P(w_t|w_{t-1},\ldots,w_{t-n+1})\right)$$


*Input:*

+ **lm**: the language model you trained (the object you returned from the `train_ngram_lm` function)
+ **data**: test data
+ **vocab**: the list of unique word types in the training set
+ **order**: order of the lm

*Output:*

+ the perplexity of test data

*Hint:*

+ If the history is not in the **lm** object, back-off to (n-1) order history to check if it is in **lm**. If no history can be found, just use `1/|V|` where `|V|` is the size of vocabulary.

In [7]:
def compute_perplexity(lm: dict[str, dict[str, float]], data: list[str], vocab: list[str], n: int = 3) -> float:
    """
    Computes perplexity using backoff logic for an n-gram language model.

    Parameters:
    -----------
    lm  : dict of {str: dict of {str: float}}
        The language model returned by train_ngram_lm (with backoff counts).
        - Keys are space-separated history strings (n - 1 context).
        - Values are dictionaries mapping next words to probabilities.
    data  : list of tokens (test data).
    vocab : list of unique tokens from training.
    n     : int, the original n (e.g. 3 => trigram). We'll internally do (n -= 1).

    Returns:
    --------
    perplexity : float
        The perplexity of 'data' under the language model 'lm'.
    """

    n -= 1  # pad
    data = (['<S>'] * n) + data
    T = 0 # accumulator for token count
    log_sum = 0.0 

    for i in range(len(data) - n):
        h, w = _backoff(lm, ' '.join(data[i : i + n])), data[i + n] # backoff (n - 1)

        # probability lookup
        prob = lm[h][w] if h in lm and w in lm[h] else 1.0 / len(vocab) # fallback to 1 / |vocab|
        log_sum += log(prob) # accumulate log prob
        T += 1

    # perplexity = exp( - (1/T) * sum of log probs )
    ppl = exp(-log_sum / T)
    return ppl

Let's evaluate the language model with different orders. You should see a decrease in perplexity as the order increases. As a reference, the perplexity of the unigram, bigram, trigram, and 4-gram language models should be around 795, 203, 141, and 130 respectively.

In [ ]:
global DATA, VOCAB

for o in [1, 2, 3, 4]:
    lm = train_ngram_lm(DATA['train'], n = o)
    print('order {} ppl {}'.format(o, compute_perplexity(lm, DATA['test'], VOCAB, n = o)))